In [1]:
import pandas as pd
import glob
from konlpy.tag import Kkma
from konlpy.utils import pprint
from tqdm import tqdm


kkma = Kkma()


existing_xlsx_list = glob.glob("./existing/*.xlsx")
emerging_xlsx_list = glob.glob("./emerging/*.xlsx")
#print (xlsx_list)

existing_dataset = {}
emerging_dataset = {}

def parse_sentence_from_dataset(dataset_path, max_len):
    intent = dataset_path.split(" ")[1].split("(")[0]
    dataset = pd.read_excel(dataset_path)
    data = dataset[["SENTENCEID", "SENTENCE", "QA"]]
    sentences = []
    cur_sentence = ""
    sentence_id = 0
    for idx, datum in data.iterrows():
        if int(datum["SENTENCEID"]) > sentence_id:
            sentence_id = int(datum["SENTENCEID"])
            next_sentence = cur_sentence + " " + str(datum["SENTENCE"])
            if len(next_sentence.split(" ")) > max_len:
                # return cur_sentence as sentence and begin new sentence with current datum
                sentences.append(cur_sentence)
                sentence_id = int(datum["SENTENCEID"])
                cur_sentence = str(datum["SENTENCE"])
            else:
                cur_sentence = next_sentence
        else:
            sentences.append(cur_sentence)
            sentence_id = int(datum["SENTENCEID"])
            cur_sentence = str(datum["SENTENCE"])
    return intent, sentences
print (f"glob.glob('./exisitng/') : {existing_xlsx_list}")

for existing_xlsx in existing_xlsx_list:
    intent, sentences = parse_sentence_from_dataset(existing_xlsx, 20)
    existing_dataset[intent] = sentences
    
for emerging_xlsx in emerging_xlsx_list:
    intent, sentences = parse_sentence_from_dataset(emerging_xlsx, 20)
    emerging_dataset[intent] = sentences

#print (existing_dataset, emerging_dataset)


glob.glob('./exisitng/') : ['./existing/E 생활서비스(11,087).xlsx', './existing/H 관광여가오락(4,949).xlsx', './existing/C 학원(4,773).xlsx', './existing/D 소매점(14,949).xlsx', './existing/B 의류(15,826).xlsx', './existing/A 음식점(15,726).xlsx', './existing/F 카페(7,859).xlsx', './existing/G 숙박업(7,113).xlsx']


In [2]:
def intent2morpheme(intent):
    result_intent = intent
    if intent == "생활서비스":
        result_intent = "생활 서비스"
    elif intent == "숙박업":
        result_intent = "숙박 업"
    elif intent == "관광여가오락":
        result_intent = "관광 여가 오락"
    elif intent == "떡집":
        result_intent = "떡 집"
    elif intent == "미술학원":
        result_intent = "미술 학원"
    elif intent == "배달음식점":
        result_intent = "배달 음식점"
    return result_intent

existing_text_file = open("./existing.txt", "w", encoding="utf-8")
emerging_text_file = open("./emerging.txt", "w", encoding="utf-8")

# collect existing intent and emerging intents

existing_text_list = []
emerging_text_list = []

for existing_intent in tqdm(existing_dataset):
    existing_contents = existing_dataset[existing_intent]
    for existing_content in tqdm(existing_contents):
        processed_existing_content = " ".join(kkma.nouns(existing_content)).strip()
        #processed_existing_content = " ".join(list(map(lambda x : x[0], kkma.pos(existing_content)))).strip()
        
        if processed_existing_content == "" or (len(processed_existing_content.split()) <= 3):
            continue
        existing_text_string = intent2morpheme(existing_intent)+ "\t" + processed_existing_content + "\n"
        existing_text_list.append(existing_text_string)
        
for emerging_intent in tqdm(emerging_dataset):
    emerging_contents = emerging_dataset[emerging_intent]
    for emerging_content in tqdm(emerging_contents):
        
        processed_emerging_content = " ".join(kkma.nouns(emerging_content)).strip()
        #processed_emerging_content = " ".join(list(map(lambda x : x[0], kkma.pos(emerging_content)))).strip()
        if processed_emerging_content == "" or (len(processed_emerging_content.split()) <= 3):
            continue
        emerging_text_string = intent2morpheme(emerging_intent)+ "\t" + processed_emerging_content + "\n"
        emerging_text_list.append(emerging_text_string)

 30%|███       | 1050/3470 [00:21<00:40, 59.19it/s]


 62%|██████▏   | 2155/3470 [00:39<00:25, 51.63it/s]


 93%|█████████▎| 3229/3470 [00:57<00:04, 53.36it/s]


 57%|█████▋    | 719/1270 [00:14<00:09, 60.65it/s]


 29%|██▉       | 514/1781 [00:08<00:22, 57.45it/s]


...

  7%|▋         | 136/2086 [00:02<00:36, 52.81it/s]


 57%|█████▋    | 1184/2086 [00:20<00:14, 61.63it/s]


 13%|█▎        | 124/928 [00:01<00:12, 65.30it/s]


 63%|██████▎   | 253/401 [00:05<00:03, 45.41it/s]


100%|██████████| 3/3 [00:26<00:00,  8.89s/it]


In [3]:
# suffle the emerging text and existing text
import random

random.shuffle(existing_text_list)
random.shuffle(emerging_text_list)

for existing_text in existing_text_list:
    existing_text_file.write(existing_text)

for emerging_text in emerging_text_list:
    emerging_text_file.write(emerging_text)


In [14]:
for k in custom_dataset:
    print (f"key :{k}, {len(custom_dataset[k])}")
    

NameError: name 'custom_dataset' is not defined